<a href="https://colab.research.google.com/github/daniel-molina23/fake-news-decision-tree/blob/daniel-develop/Decision_Tree_from_Daniel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# The modules we're going to use
from __future__ import print_function
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import randint

# Load data from scikit-learn, which returns (data, target)
# note: if you call "boston = load_boston()", it returns a dictionary-like object
data, target = datasets.load_boston(True)

# Split the data into two parts: training data and testing data
train_data,test_data,train_target,test_target = train_test_split(data,(target[:, np.newaxis]), test_size=0.2, random_state=42)




In [11]:
import pandas as pd

df1 = pd.read_csv('politifact_real.csv')
df2 = pd.read_csv("politifact_fake.csv")

poli_real = pd.DataFrame(df1)
poli_fake = pd.DataFrame(df2)

poli_fake.shape

(432, 4)

In [12]:
poli_real["label"] = 0
poli_fake["label"] = 1

poli_data = pd.concat([poli_real, poli_fake])

from sklearn.utils import shuffle
poli_data = shuffle(poli_data)

poli_data.head(20)

poli_data.dropna()

,id,news_url,title,tweet_ids,label
205,politifact14095,https://usawatchdog.com/clinton-foundation-lar...,Clinton Foundation Largest Unprosecuted Charit...,811287224163635200\t811289148925411328\t811289...,1
192,politifact14085,usleader.net/index.php/2017/05/11/breaking-bil...,BREAKING: Bill O’Reilly In Critical Condition ...,860131864794861569\t860136023279403009\t860173...,1
166,politifact14187,http://rip.trendolizer.com/2017/06/breaking-mo...,BREAKING: Monica Lewinsky Found Dead In Roslin...,871031716277882880\t871032109422567425\t871051...,1
341,politifact12120,http://politicaladarchive.org/ad/PolAd_BernieS...,Political TV Ad Archive » PolAd,425654414473707521\t425675162600472576\t425700...,0
77,politifact14506,https://yournewswire.com/tens-of-thousands-of-...,Tens of Thousands Of Scientists Declare Climat...,771610790285553664\t771641422017548288\t771648...,1
...,...,...,...,...,...
117,politifact13747,https://web.archive.org/web/20170205045142/htt...,Bill Gates: “I think Donald Trump will go down...,827982406913241088\t827983104602091520\t827983...,1
393,politifact13900,https://twitter.com/RepMaloney/status/84242248...,"Carolyn B. Maloney on Twitter: ""Women's health...",1002880963158986753\t1002884280526270468\t1002...,0
347,politifact2724,http://data.bls.gov/PDQ/servlet/SurveyOutputSe...,Notice: Data not available: U.S. Bureau of Lab...,264445210791129088,0
289,politifact14815,https://www.helloforos.com/t/miss-universe-gua...,Miss Universe Guayana 2016 arrestada con 2kilo...,833467828523110400\t833481837997654017\t833487...,1


In [13]:
poli_data.head(30)
# com, co, gov, etc (domain parser)
# https, http, or neither (https is far more secure!)
# number of tweet id's per link
# (future, not now) using web scraping techniques to find number of advertisements per link
# poli_data.iloc[0][4]

,id,news_url,title,tweet_ids,label
205,politifact14095,https://usawatchdog.com/clinton-foundation-lar...,Clinton Foundation Largest Unprosecuted Charit...,811287224163635200\t811289148925411328\t811289...,1
192,politifact14085,usleader.net/index.php/2017/05/11/breaking-bil...,BREAKING: Bill O’Reilly In Critical Condition ...,860131864794861569\t860136023279403009\t860173...,1
166,politifact14187,http://rip.trendolizer.com/2017/06/breaking-mo...,BREAKING: Monica Lewinsky Found Dead In Roslin...,871031716277882880\t871032109422567425\t871051...,1
163,politifact13484,https://www.washingtonpost.com/graphics/politi...,Thinking about writing in a candidate on Elect...,NaN,1
341,politifact12120,http://politicaladarchive.org/ad/PolAd_BernieS...,Political TV Ad Archive » PolAd,425654414473707521\t425675162600472576\t425700...,0
77,politifact14506,https://yournewswire.com/tens-of-thousands-of-...,Tens of Thousands Of Scientists Declare Climat...,771610790285553664\t771641422017548288\t771648...,1
423,politifact13475,genius.com/discussions/219349-Donald-trump-pro...,Donald Trump Protester Speaks Out: “I Was Paid...,739971101288239104\t739984202116960256\t740014...,1
145,politifact5469,http://www.youtube.com/watch?v=TuYCknrJZ7A,White and Blue Fund ad,147369605797642242\t147369981032660992\t147378...,0
360,politifact14499,http://theeconomiccollapseblog.com/archives/hu...,Hurricane Irma: If There Was Such A Thing As A...,905750295820591105,1
185,politifact95,http://findarticles.com/p/articles/mi_qn4155/i...,FindArticles.com,NaN,0


In [47]:
# here is where web scraping is performed!
import re
import requests

# regexProtocol = re.compile(r'(http|https)?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
# regexNoProtocol = re.compile(r'[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')
regexUrl = re.compile(r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})')
urls = poli_data["news_url"]


# just for testing, uncommmment to test!
check = [
  'http://www.foufos.us',
  'https://www.foufos.gov',
  'http://foufos.gr',
  'http://www.foufos.gr/kino',
  'http://werer.uk',
  'www.foufos.org',
  'www.mp3.com',
  'www.t.co',
  'http://t.co',
  'http://www.t.co',
  'https://www.t.co',
  'www.aa.com',
  'http://aa.com',
  'http://www.aa.com',
  'https://www.aa.com',
  'www.foufos',
  'www.foufos-.gr',
  'www.-foufos.gr',
  'foufos.gr',
  'http://www.foufos',
  'http://foufos',
  'www.mp3#.com'
]

for each in check:
  if regexUrl.match(each):
    print("SUCESS:::::", each)
  else:
    print("FAILURE---------------------   ", each)

# arr = []
# for url in urls:
#   if isinstance(url, float):
#     arr.append(-1) # not valid url, no urls
#     continue # go to next iteration
#   elif url

SUCESS::::: http://www.foufos.us
SUCESS::::: https://www.foufos.gov
SUCESS::::: http://foufos.gr
SUCESS::::: http://www.foufos.gr/kino
SUCESS::::: http://werer.uk
SUCESS::::: www.foufos.org
SUCESS::::: www.mp3.com
SUCESS::::: www.t.co
SUCESS::::: http://t.co
SUCESS::::: http://www.t.co
SUCESS::::: https://www.t.co
SUCESS::::: www.aa.com
SUCESS::::: http://aa.com
SUCESS::::: http://www.aa.com
SUCESS::::: https://www.aa.com
FAILURE---------------------    www.foufos
FAILURE---------------------    www.foufos-.gr
FAILURE---------------------    www.-foufos.gr
FAILURE---------------------    foufos.gr
FAILURE---------------------    http://www.foufos
FAILURE---------------------    http://foufos
FAILURE---------------------    www.mp3#.com


In [14]:
# most common domains (45 total)
domain = [".com/", ".org/", ".edu/", ".gov/", ".uk/", ".net/", ".ca/", ".de/", ".jp/", ".fr/", ".au/", ".us/", ".ru/", ".ch/", ".it/", ".nl/", ".se/", ".no/", ".es/", ".mil/", ".ly/", ".tel/", ".kitchen/", ".email/", ".tech/", ".estate/", ".xyz/", ".codes/", ".bargains/", ".bid/", ".expert/", ".co/", ".name/", ".mobi/", ".asia/", ".biz/", ".arpa/", ".cat/", ".jobs/", ".info/", ".int/", ".pro/", ".aero/", ".travel/", ".coop/"]
d_encoding = [index for index in range(len(domain))] # encoded by index
print(len(domain) == len(d_encoding))

# Protocols for encryption
protocol = ["no_protocol", "http", "https"]
p_encoding = [0, 1, 2]

# Number of tweet id's per row


True


In [15]:
# data_domain method
def encodeDomain(link):
  num = len(domain) # none of the domains encoding, default
  if isinstance(link, float): # nan, therefore return
    return num
  for i in range(len(domain)):
    if domain[i] in link:
      num = i
  return num

# data_protocol method
def configProtocol(link):
  if isinstance(link, float): # nan, therefore return
    return 0
  elif link[:5] == "https":
    return 2
  elif link[:4] == "http":
    return 1
  else:
    return 0


# number of tweet id's method
def getIdCount(idCombo):
  if isinstance(ids, float): # if 'nan'
    return 0
  return len(idCombo.split('\t')) # split by tab or by the char '\t'


####################

data_protocol = []
data_domain = []
for link in poli_data["news_url"]:
  data_domain.append(encodeDomain(link)) # returns domain encoding
  data_protocol.append(configProtocol(link)) # returns either 'https', 'http', or 'neither'


tweetIds = []
for ids in poli_data["tweet_ids"]:
  tweetIds.append(getIdCount(ids))



label = []
for l in poli_data["label"]:
  label.append(l)


# are all strings the same length????
print(len(data_domain) == len(data_protocol) == len(tweetIds) == len(label))


True


In [16]:
# data_domain, all numbers
# d_encoding (indices of domain)
# domain (list of string domains)
d_freq = dict()
d_set = set(domain)
for i in data_domain:
  if i == len(domain):
    if "NULL" not in d_freq:
      d_freq["NULL"] = 1
    else:
      d_freq["NULL"] += 1
    continue # go to next iteration

  if domain[i] not in d_freq:
    d_freq[domain[i]] = 1
  else: # valid
    d_freq[domain[i]] += 1

display(d_freq)

{'.co/': 4,
 '.com/': 571,
 '.edu/': 4,
 '.gov/': 119,
 '.info/': 7,
 '.mil/': 1,
 '.net/': 17,
 '.org/': 217,
 '.ru/': 1,
 '.uk/': 6,
 '.us/': 12,
 '.xyz/': 1,
 'NULL': 96}

In [17]:
clean_data = pd.DataFrame({
    "domain_type":data_domain,
    "protocol": data_protocol,
    "tweet_ids": tweetIds,
    "label": label
})

clean_data.head(10)

,domain_type,protocol,tweet_ids,label
0,0,2,291,1
1,5,0,21,1
2,0,1,40,1
3,0,2,0,1
4,1,1,1025,0
5,0,2,1893,1
6,0,0,1025,1
7,0,1,208,0
8,0,1,1,1
9,0,1,0,0


In [18]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

In [19]:
features = ["domain_type", "protocol", "tweet_ids"]
X = clean_data[features]
y = clean_data.label

display(X.head())
display(y.head())

,domain_type,protocol,tweet_ids
0,0,2,291
1,5,0,21
2,0,1,40
3,0,2,0
4,1,1,1025


0    1
1    1
2    1
3    1
4    0
Name: label, dtype: int64

In [20]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1) # 80% training and 20% test

In [21]:
clf = DecisionTreeClassifier()

# train decision tree
clf = clf.fit(X_train, y_train)

# predict
y_pred = clf.predict(X_test)

In [22]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7452830188679245


#Using another type of decision Tree classifier

In [23]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=8) # the greater the max_depth, can cause overfitting

# Train DT
clf = clf.fit(X_train,y_train)

#Predict response for test dataset
y_pred = clf.predict(X_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7830188679245284
